In [1]:
import sys
sys.path.append('/scratch/pradap/python-work/anhaidgroup/py_entitymatching/')

In [13]:
import py_entitymatching as em
import pandas as pd

In [3]:
import mur_utils as mu

In [4]:
A = em.read_csv_metadata('ORIG_DBLP_ACM/dblp_mur_clean.csv', key='id', dtype=str)
B = em.read_csv_metadata('ORIG_DBLP_ACM/acm_mur_clean.csv',  key='id', dtype=str)

In [5]:
# load gold
gold = em.read_csv_metadata('ORIG_DBLP_ACM/goldWithLabel.csv', key='_id', fk_ltable='ltable_id', fk_rtable='rtable_id',
                           ltable=A, rtable=B, dtype=str)

In [6]:
count, a_indices, b_indices = mu.get_num_positives_tables(A, B, gold)

In [7]:
ab = em.AttrEquivalenceBlocker()
C1 = ab.block_tables(A, B, 'year', 'year', ['title', 'authors', 'year'], ['title', 'authors', 'year'])

In [8]:
dbg = em.debug_blocker(C1, A, B)

In [16]:
len(pd.unique(dbg['ltable_id'])), len(pd.unique(dbg['rtable_id']))

(179, 178)

In [23]:
A1 = A.set_index('id', drop=False)
B1 = B.set_index('id', drop=False)

In [24]:
len(A1), len(B1)

(1979, 1991)

In [25]:
A2 = A1.ix[(set(A1.index.values).difference(pd.unique(dbg['ltable_id'])))]
B2 = B1.ix[(set(B1.index.values).difference(pd.unique(dbg['rtable_id'])))]

In [27]:
len(A2), len(B2)

(1800, 1813)

In [28]:
count, a_indices, b_indices = mu.get_num_positives_tables(A2, B2, gold)

In [29]:
print(count)

1575


In [30]:
em.set_key(A2, 'id')
em.set_key(B2, 'id')

True

In [34]:
A2.reset_index(inplace=True, drop=True)
B2.reset_index(inplace=True, drop=True)

In [35]:
em.to_csv_metadata(A2, 'ORIG_DBLP_ACM/dblp_mur_diff_year_sim_title_removed.csv')
em.to_csv_metadata(B2, 'ORIG_DBLP_ACM/acm_mur_diff_year_sim_title_removed.csv')

True

In [37]:
min(A1.year)

'1994'

In [38]:
ab = em.AttrEquivalenceBlocker()
C2 = ab.block_tables(A2, B2, 'year', 'year', ['title', 'authors', 'year'], ['title', 'authors', 'year'])

In [40]:
dbg = em.debug_blocker(C2, A2, B2)

In [41]:
em.view_table(dbg)

In [43]:
count, a_indices, b_indices = mu.get_num_positives_tables(A2, B2, gold)
print(count)

1575


In [44]:
import numpy as np

In [61]:
indices = list(np.random.choice(list(b_indices), 200))

In [62]:
B3 = B2.set_index('id', drop=False)

In [63]:
B4 = mu.perturb_year(B3, indices, 2003, 1994)

[index: 344869] old: 1999
new: 2000
[index: 381879] old: 1996
new: 1997
[index: 872814] old: 2003
new: 2003
[index: 765556] old: 1997
new: 1998
[index: 276392] old: 1998
new: 1999
[index: 290596] old: 1998
new: 1999
[index: 673144] old: 1995
new: 1996
[index: 672020] old: 2000
new: 2001
[index: 227624] old: 1996
new: 1997
[index: 615186] old: 1994
new: 1995
[index: 276346] old: 1998
new: 1999
[index: 673656] old: 1997
new: 1998
[index: 615194] old: 1994
new: 1995
[index: 671330] old: 1998
new: 1999
[index: 253368] old: 1997
new: 1998
[index: 765528] old: 1998
new: 1999
[index: 273271] old: 1998
new: 1999
[index: 671670] old: 1999
new: 2000
[index: 375757] old: 2001
new: 2002
[index: 352982] old: 2000
new: 2001
[index: 234900] old: 1996
new: 1997
[index: 369276] old: 2000
new: 2001
[index: 276347] old: 1998
new: 1999
[index: 234900] old: 1997
new: 1998
[index: 872777] old: 2003
new: 2003
[index: 672181] old: 2001
new: 2002
[index: 673137] old: 1995
new: 1996
[index: 767148] old: 2001
ne

In [64]:
em.set_key(B4, 'id')

True

In [98]:
ab = em.AttrEquivalenceBlocker()
C2 = ab.block_tables(A2, B4, 'year', 'year', ['title', 'authors', 'year'], ['title', 'authors', 'year'])

In [99]:
dbg = em.debug_blocker(C2, A2, B4)

In [100]:
em.view_table(dbg)

In [68]:
B4.reset_index(inplace=True, drop=True)

In [69]:
B4.head()

,id,title,authors,venue,year
0,304201,An efficient bitmap encoding scheme for selection queries,"Chee-Yong Chan, Yannis E. Ioannidis",International Conference on Management of Data,1999
1,672964,Integrating a Structured-Text Retrieval System with an Object-Oriented Database System,"Tak W. Yan, Jurgen Annevelink",Very Large Data Bases,1994
2,223794,Nearest neighbor queries,"Nick Roussopoulos, Stephen Kelley, Frédéric Vincent",International Conference on Management of Data,1995
3,223788,The LyriC language: querying constraint objects,"Alexander Brodsky, Yoram Kornatzky",International Conference on Management of Data,1995
4,672012,E.piphany Epicenter Technology Overview,Sridhar Ramaswamy,Very Large Data Bases,2000


In [71]:
em.to_csv_metadata(B4, './ORIG_DBLP_ACM/acm_sim_title_rem_perturb_year.csv')

True

In [101]:
ob = em.OverlapBlocker()
C3 = ob.block_tables(A2, B4, 'title', 'title', l_output_attrs=['title', 'authors', 'year'], r_output_attrs=['title', 'authors', 'year'])

0%                          100%
[##############################] | ETA: 00:00:00
Total time elapsed: 00:00:01


In [102]:
C = em.combine_blocker_outputs_via_union([C2, C3])

In [103]:
count, tuple_ids = mu.get_num_positives_candset(C, gold)

In [104]:
count

1575

In [204]:
g = gold.set_index(['ltable_id', 'rtable_id'], drop=False)

In [208]:
g1 = g.ix[tuple_ids]

In [210]:
em.set_fk_ltable(g1, 'ltable_id')
em.set_fk_rtable(g1, 'rtable_id')
em.set_ltable(g1, A2)
em.set_rtable(g1, B4)
em.set_key(g1, '_id')


True

In [215]:
g1.reset_index(drop=True, inplace=True)

In [216]:
g1 = em.add_output_attributes(g1, l_output_attrs=['year'], r_output_attrs=['year'])

In [217]:
g1.head()

,_id,ltable_id,rtable_id,label,ltable_year,rtable_year
0,769,conf/sigmod/FlorescuGKR02,564779,1,2002,2002
1,653,conf/sigmod/WangPFY03,872774,1,2003,2003
2,174,conf/sigmod/AggarwalWY99,304218,1,1999,1999
3,342,conf/sigmod/LlirbatFS97,253357,1,1997,1997
4,234,conf/sigmod/ZhouLP99,304580,1,1999,1999


In [218]:
g1.rename(columns={'rtable_year': 'rtable_year_perturbed'}, inplace=True)

In [219]:
em.set_fk_ltable(g1, 'ltable_id')
em.set_fk_rtable(g1, 'rtable_id')
em.set_ltable(g1, A2)
em.set_rtable(g1, B2)
em.set_key(g1, '_id')

True

In [221]:
g1 = em.add_output_attributes(g1, r_output_attrs=['year'])

In [222]:
g1.head()

,_id,ltable_id,rtable_id,label,ltable_year,rtable_year_perturbed,rtable_year
0,769,conf/sigmod/FlorescuGKR02,564779,1,2002,2002,2002
1,653,conf/sigmod/WangPFY03,872774,1,2003,2003,2003
2,174,conf/sigmod/AggarwalWY99,304218,1,1999,1999,1999
3,342,conf/sigmod/LlirbatFS97,253357,1,1997,1997,1997
4,234,conf/sigmod/ZhouLP99,304580,1,1999,1999,1999


In [224]:
g1_same_year = em.filter_rows(g1, 'ltable_year == rtable_year_perturbed')
len(g1_same_year)

1396

In [225]:
g1_diff_year = em.filter_rows(g1, 'ltable_year != rtable_year_perturbed')
len(g1_diff_year)

179

In [226]:
g1_diff_year_sampled = em.sample_table(g1_diff_year, 100)

In [227]:
g1_same_year_splitted_2 = em.split_train_test(g1_same_year, train_proportion=0.5)

In [228]:
g1_same_year_to_be_swapped_sample = em.sample_table(g1_same_year_splitted_2['train'], 80)
g1_same_year_not_to_be_swapped_sample = em.sample_table(g1_same_year_splitted_2['test'], 120)

In [229]:
len(g1_same_year_to_be_swapped_sample), len(g1_same_year_not_to_be_swapped_sample)

(80, 120)

In [231]:
g1_diff_year_sampled.head(10)

,_id,ltable_id,rtable_id,label,ltable_year,rtable_year_perturbed,rtable_year
15,1622,conf/vldb/MengLYWCR98,671185,1,1998,1999,1998
27,68,conf/sigmod/IpeirotisGS01,375671,1,2001,2002,2001
54,2080,journals/vldb/HarrisR96,765755,1,1996,1995,1996
59,402,conf/sigmod/ChristophidesCM96,233358,1,1996,1995,1996
61,1816,conf/vldb/VermeerA96,673635,1,1996,1997,1996
70,1605,conf/vldb/KeimH99,671505,1,1999,2000,1999
74,2137,journals/vldb/HelmerM03,950493,1,2003,2002,2003
121,704,conf/sigmod/LuoEHN02,564721,1,2002,2001,2002
127,113,conf/sigmod/LiuPBHPT00,336579,1,2000,2001,2000
146,1711,conf/vldb/RothS97,670992,1,1997,1996,1997


In [232]:
len(g1_same_year_to_be_swapped_sample.ltable_id.values)

80

In [242]:
A3 = A2.set_index('id', drop=False)

In [243]:
titles = A3.ix[g1_same_year_to_be_swapped_sample.ltable_id.values, 'title']

In [245]:
titles.head()

id
conf/vldb/Rengarajan96                                        How System 11 SQL Server Became Fast
conf/sigmod/JagadishJOT01                                        Global Optimization of Histograms
journals/sigmod/PuSW01        Infosphere Project: System Support for Information Flow Applications
journals/sigmod/HanBP01                                                 Wrapping Web Data into XML
journals/sigmod/SommerGB02                                               The n-Tier Hub Technology
Name: title, dtype: object

In [246]:
authors = A3.ix[g1_same_year_to_be_swapped_sample.ltable_id.values, 'authors']

In [247]:
A4 = A3.copy()

In [248]:
A4.ix[g1_same_year_to_be_swapped_sample.ltable_id.values, 'authors'] = titles

In [249]:
A4.ix[g1_same_year_to_be_swapped_sample.ltable_id.values, 'title'] = authors

In [251]:
A4.ix[g1_same_year_to_be_swapped_sample.ltable_id.values].head()

,id,title,authors,venue,year
id,,,,,
conf/vldb/Rengarajan96,conf/vldb/Rengarajan96,T. K. Rengarajan,How System 11 SQL Server Became Fast,VLDB,1996
conf/sigmod/JagadishJOT01,conf/sigmod/JagadishJOT01,"H. V. Jagadish, Kian-Lee Tan, Hui Jin, Beng Chin Ooi",Global Optimization of Histograms,SIGMOD Conference,2001
journals/sigmod/PuSW01,journals/sigmod/PuSW01,"Calton Pu, Jonathan Walpole, Karsten Schwan",Infosphere Project: System Support for Information Flow Applications,SIGMOD Record,2001
journals/sigmod/HanBP01,journals/sigmod/HanBP01,"David Buttler, Wei Han, Calton Pu",Wrapping Web Data into XML,SIGMOD Record,2001
journals/sigmod/SommerGB02,journals/sigmod/SommerGB02,"Rainer A. Sommer, David Bailey, Thomas R. Gulledge",The n-Tier Hub Technology,SIGMOD Record,2002


In [252]:
em.set_key(A4, 'id')

True

In [253]:
A4.reset_index(inplace=True, drop=True)

In [254]:
A4.head()

,id,title,authors,venue,year
0,conf/sigmod/Team95a,Paradise: A Database System for GIS Applications,Paradise Team,SIGMOD Conference,1995
1,conf/sigmod/BunemanDHS96,A Query Language and Optimization Techniques for Unstructured Data,"Gerd G. Hillebrand, Peter Buneman, Susan B. Davidson, Dan Suciu",SIGMOD Conference,1996
2,conf/sigmod/ShenoyHSBBS00,Turbo-charging Vertical Mining of Large Databases,"Jayant R. Haritsa, Devavrat Shah, S. Sudarshan, Pradeep Shenoy, Mayank Bawa, Gaurav Bhalotia",SIGMOD Conference,2000
3,conf/sigmod/MumickQM97,Maintenance of Data Cubes and Summary Tables in a Warehouse,"Inderpal Singh Mumick, Dallan Quass, Barinderpal Singh Mumick",SIGMOD Conference,1997
4,conf/sigmod/ChaudhuriKN03,On Relational Support for XML Publishing: Beyond Sorting and Tagging,"Raghav Kaushik, Jeffrey F. Naughton, Surajit Chaudhuri",SIGMOD Conference,2003


In [255]:
em.to_csv_metadata(A4, 'ORIG_DBLP_ACM/dblp_sim_title_rem_title_auth_swapped.csv')

True

In [256]:
A4.head()

,id,title,authors,venue,year
0,conf/sigmod/Team95a,Paradise: A Database System for GIS Applications,Paradise Team,SIGMOD Conference,1995
1,conf/sigmod/BunemanDHS96,A Query Language and Optimization Techniques for Unstructured Data,"Gerd G. Hillebrand, Peter Buneman, Susan B. Davidson, Dan Suciu",SIGMOD Conference,1996
2,conf/sigmod/ShenoyHSBBS00,Turbo-charging Vertical Mining of Large Databases,"Jayant R. Haritsa, Devavrat Shah, S. Sudarshan, Pradeep Shenoy, Mayank Bawa, Gaurav Bhalotia",SIGMOD Conference,2000
3,conf/sigmod/MumickQM97,Maintenance of Data Cubes and Summary Tables in a Warehouse,"Inderpal Singh Mumick, Dallan Quass, Barinderpal Singh Mumick",SIGMOD Conference,1997
4,conf/sigmod/ChaudhuriKN03,On Relational Support for XML Publishing: Beyond Sorting and Tagging,"Raghav Kaushik, Jeffrey F. Naughton, Surajit Chaudhuri",SIGMOD Conference,2003


In [257]:
dbg = em.debug_blocker(C, A2, B4)

In [258]:
len(dbg)

200

In [259]:
em.view_table(dbg)

In [260]:
d1 = dbg[0:150]

In [261]:
len(d1)

150

In [262]:
g3.head()

,_id,ltable_id,rtable_id,label,ltable_year,rtable_year_perturbed,rtable_year
0,362,conf/sigmod/HanKK97,253330,1,1997,1997,1997
2,432,conf/sigmod/PoosalaIHS96,233342,1,1996,1996,1996
13,126,conf/sigmod/KornM00,335415,1,2000,2000,2000
16,825,journals/sigmod/dOnofrioP03,776990,1,2003,2003,2003
20,1897,conf/vldb/AgrawalPWZ95,673157,1,1995,1995,1995


In [263]:
d2 = d1[['_id', 'ltable_id', 'rtable_id']]

In [267]:
len(g1_diff_year_sampled), len(g1_same_year_not_to_be_swapped_sample), len(g1_same_year_to_be_swapped_sample), len(d2)

(100, 120, 80, 150)

In [268]:
g1_diff_year_sampled_prj = g1_diff_year_sampled[['_id', 'ltable_id', 'rtable_id', 'label']]
g1_same_year_not_to_be_swapped_sample = g1_same_year_not_to_be_swapped_sample[['_id', 'ltable_id', 'rtable_id', 'label']]
g1_same_year_to_be_swapped_sample = g1_same_year_to_be_swapped_sample[['_id', 'ltable_id', 'rtable_id', 'label']]

In [270]:
d2['label'] = 0

/scratch/pradap/local/share/miniconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [272]:
labeled_data = pd.concat([g1_diff_year_sampled_prj, g1_same_year_not_to_be_swapped_sample, g1_same_year_to_be_swapped_sample, d2], 
          ignore_index=True)

In [276]:
sum(labeled_data.label.astype(int))

300

In [279]:
labeled_data = labeled_data.sample(frac=1)

In [281]:
labeled_data['_id'] = range(0, len(labeled_data))

In [282]:
labeled_data.reset_index(inplace=True)

In [283]:
labeled_data.head()

,index,_id,ltable_id,rtable_id,label
0,411,0,journals/sigmod/LabrinidisR00,335430,0
1,280,1,conf/vldb/CarusoCGLM00,671870,1
2,107,2,conf/vldb/Toivonen96,673325,1
3,321,3,conf/vldb/AilamakiDHS01,671662,0
4,297,4,conf/sigmod/BrunoC02,564722,1


In [284]:
labeled_data = labeled_data[['_id', 'ltable_id', 'rtable_id', 'label']]

In [286]:
em.set_key(labeled_data, '_id')
em.set_ltable(labeled_data, A4)
em.set_rtable(labeled_data, B4)
em.set_fk_ltable(labeled_data, 'ltable_id')
em.set_fk_rtable(labeled_data, 'rtable_id')

True

In [287]:
em.to_csv_metadata(labeled_data, 'ORIG_DBLP_ACM/labeled_data_dblp_acm_after_monkey_tricks.csv')

True